## Семинар 7: Loguru. BS. Requests. Wandb.

## Логирование, библиотека Loguru:

Loguru is a library which aims to bring enjoyable logging in Python.

In [ ]:
from loguru import logger

logger.debug("That's it, beautiful and simple logging!")

In [ ]:
logger.add("out.log", backtrace=True, diagnose=True)  # Caution, may leak sensitive data in prod

def func(a, b):
    return a / b

def nested(c):
    try:
        func(5, c)
    except ZeroDivisionError:
        logger.exception("What?!")

nested(0)

Рассмотрим подробнее в [документации](https://github.com/Delgan/loguru)

### Извлечение данных из ```xml``` и ```html``` файлов. Библиотека Beautiful Soup:

Начнем с искусственного примера:

In [ ]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

Парсим html данные:

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

Теперь напечатаем код странички с форматированием:

In [ ]:
print(soup.prettify())

Библиотека позволяет перемещаться по DOM дереву и получать нужные нам элементы:

### ```title```

In [ ]:
print(soup.title)

### ```title.name```

In [ ]:
print(soup.title.name)

### ```title.string```

In [ ]:
print(soup.title.string)

### ```title.parent.name```

In [ ]:
print(soup.title.parent.name)

### ```p```

In [ ]:
print(soup.p)

### ```p['class']```

In [ ]:
print(soup.p['class'])

### ```a```

In [ ]:
print(soup.a)

### ```find_all```

In [ ]:
for a in soup.find_all('a'):
    print(a)

### ```find```

In [ ]:
print(soup.find(id="link3"))

### Пример парсинга данных из сети:

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

[Requests](Requests) is an elegant and simple HTTP library for Python, built for human beings.

Requests позволяет очень легко отправлять HTTP/1.1 запросы, без необходимости  вручную добавлять query строки к своим URL-адресам или кодировать данные POST. 

In [ ]:
requests.get("http://rbc.ru/")

Можем посмотреть всю сопутствующую запросу информацию:

In [ ]:
%%time
resp=requests.get("https://lenta.ru/brief/2019/03/25/apple/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

И сам текст:

In [ ]:
resp.text[:1000]

In [ ]:
BeautifulSoup(resp.text, "html.parser").get_text()[:1000]

Разбиваем по параграфам и получаем нужный результат:

In [ ]:
bs=BeautifulSoup(resp.text, "html.parser")
title=bs.h1.text
text=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html.parser").get_text()
print(title, "\n-----\n", text)

## Логирование/отслеживание экспериментов (Experiment tracking). Weights & Biases.


Чтобы результаты записывались в облако, нужно залогиниться. Предоставить свой api-key. Самый простой способ - использование консоли:
```bash 
$ wandb login
```

In [ ]:
import wandb
wandb.login()

Посмотрим на следующий псевдо-эксперимент:

In [ ]:
import math
import random

# Start a new run, tracking hyperparameters in config
wandb.init(project="test-drive", config={
    "learning_rate": 0.01,
    "dropout": 0.2,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
})
config = wandb.config

# Simulating a training or evaluation loop
for x in range(50):
    acc = math.log(1 + x + random.random()*config.learning_rate) + random.random() + config.dropout
    loss = 10 - math.log(1 + x + random.random() + config.learning_rate*x) + random.random() + config.dropout
    # Log metrics from your script to W&B
    wandb.log({"acc":acc, "loss":loss})

По ссылке выше можно смотреть результаты. Данные по одному проекту собираются вместе и их можно сравнивать. Запустите ячейку с этим экспериментов еще один раз и посмотрите, что произойдет.

Разберем еще несколько примеров:

In [ ]:
# Simple Keras Model

import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from wandb.keras import WandbCallback

# Set an experiment name to group training and evaluation
experiment_name = wandb.util.generate_id()

# Start a run, tracking hyperparameters
wandb.init(
  project="intro-demo",
  group=experiment_name,
  config={
    "layer_1": 512,
    "activation_1": "relu",
    "dropout": 0.2,
    "layer_2": 10,
    "activation_2": "softmax",
    "optimizer": "sgd",
    "loss": "sparse_categorical_crossentropy",
    "metric": "accuracy",
    "epoch": 50,
    "batch_size": 32
  })
config = wandb.config

# Get the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
])
model.compile(optimizer=config.optimizer,
              loss=config.loss,
              metrics=[config.metric]
              )

history = model.fit(x=x_train,
                    y=y_train,
                    epochs=config.epoch,
                    batch_size=config.batch_size,
                    validation_data=(x_test, y_test),
                    # Use the WandbCallback to automatically save all the metrics tracked in model.fit() to your dashboard
                    callbacks=[WandbCallback()])

Можем дополнительно поработать с результатами локально и сохранить их в wandb:

In [ ]:
!pip install scikit-plot -qqq

import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

wandb.init(project="intro-demo", group=experiment_name)

# Log F1 Score
y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)
f1 = f1_score(y_test, y_test_pred_class, average='micro')
wandb.log({"f1": f1}, commit=False)

# Log Confusion Matrix
fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
wandb.log({"confusion_matrix": wandb.Image(fig)}, commit=False)

# Log ROC Curve
fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
wandb.log({"plot_roc": wandb.Image(fig)}, commit=True)

class_score_data = []
for test, pred in zip(y_test, y_test_pred):
      class_score_data.append([test, pred])
wandb.log({"class_scores": wandb.Table(data=class_score_data,
                                           columns=["test", "pred"])})

### Лучшие практики:


Projects: Логируйте несколько запусков в один проект, чтобы затем сравнить их. ```wandb.init(project="project-name")```

Groups: Используйте группы, для группировки нескольки процессов одного эксперимента. ```wandb.init(group='experiment-1')```

Tags: Используйте тэги для индикации лучшей модели или текущего бейслайна.

Notes: Записывайте дополнительные комментарий к каждому запуску. 

Reports: Создавайте отчеты по группам экспериментов. 


### Дополнительные возможности:

1. Определение [переменных окружения](https://docs.wandb.com/library/environment-variables) (Environment variables). 

2. [Offline mode](https://docs.wandb.com/library/technical-faq#can-i-run-wandb-offline): Используйте `dryrun` режим, для тренировки офлайн, а синхронизацию можно будет сделать позже.

3. [Sweeps](https://docs.wandb.com/sweeps): Позволяют подбирать гиперпараметры просто и быстро.

5. [Artifacts](https://docs.wandb.com/artifacts): Позволяет контролить версии моделей и датасетов.

На этом возможности не ограничиваются. Примеры можно найти в галерее: https://wandb.ai/gallery. 

### Задание 1: Зарегистрируйтесь в wandb. Запустите один из экспериментов выше. Добавьте дополнительные переменные. Постройте графики к ним. Создайте отчет. 

### Задание 2*: Запустите Sweeps для подбора гиперпараметров.